# 02. DATA PREPROCESSING

In [58]:
import pandas as pd

file_path = 'data/raw/hcm_weather.csv'
df = pd.read_csv(file_path)

## 1. Xử lý cơ bản dữ liệu

In [59]:
# Số dòng/cột
print("Số dòng:", df.shape[0])
print("Số dòng:", df.shape[1])

Số dòng: 128448
Số dòng: 11


In [60]:
# xem qua dữ liệu
df.head(10)

,latitude,longitude,datetime,temperature (C degree),humidity (%),precipitation (mm),windspeed (km/h),winddirection (degrees),cloudcover (%),pressure (hPa),dewpoint (C degree)
0,10.775388,106.702825,2024-01-01T00:00,25.6,88,0.0,5.7,145,99,1011.5,23.4
1,10.775388,106.702825,2024-01-01T01:00,25.3,90,0.0,3.6,143,96,1011.2,23.5
2,10.775388,106.702825,2024-01-01T02:00,25.3,88,0.0,0.5,135,96,1010.7,23.2
3,10.775388,106.702825,2024-01-01T03:00,24.9,91,0.0,4.7,9,98,1010.5,23.3
4,10.775388,106.702825,2024-01-01T04:00,24.6,92,0.0,8.0,352,100,1010.4,23.2
5,10.775388,106.702825,2024-01-01T05:00,24.4,92,0.0,7.1,345,99,1010.9,23.0
6,10.775388,106.702825,2024-01-01T06:00,24.2,92,0.0,6.3,347,98,1011.2,22.8
7,10.775388,106.702825,2024-01-01T07:00,24.5,90,0.0,5.2,348,98,1011.8,22.7
8,10.775388,106.702825,2024-01-01T08:00,26.2,78,0.0,7.1,345,99,1013.0,22.1
9,10.775388,106.702825,2024-01-01T09:00,28.1,68,0.0,6.3,336,100,1013.6,21.6


In [61]:
# Các cột và định dạng từng cột:
cols = df.columns
print(df.dtypes)

latitude                   float64
longitude                  float64
datetime                    object
temperature (C degree)     float64
humidity (%)                 int64
precipitation (mm)         float64
windspeed (km/h)           float64
winddirection (degrees)      int64
cloudcover (%)               int64
pressure (hPa)             float64
dewpoint (C degree)        float64
dtype: object


Các cột có kiểu dữ liệu phù hợp, chỉ có cột 'datetime' đang là kiểu object thay vì kiểu datetime. Ta cần định dạng lại cột 'datetime'

In [62]:
# Định dạng datetime từ kiểu object thành kiểu datetime
df['datetime'] = pd.to_datetime(df['datetime'])
print(df['datetime'].dtypes)

datetime64[ns]


In [63]:
# Số dòng thiếu ở mỗi cột:
print(df.isnull().sum())

latitude                   0
longitude                  0
datetime                   0
temperature (C degree)     0
humidity (%)               0
precipitation (mm)         0
windspeed (km/h)           0
winddirection (degrees)    0
cloudcover (%)             0
pressure (hPa)             0
dewpoint (C degree)        0
dtype: int64


In [64]:
# Số dòng trùng lặp
duplicate_count = df.duplicated().sum()
print(f"Số dòng trùng lặp: {duplicate_count}")

Số dòng trùng lặp: 0


In [65]:
# Khoảng giá trị của các cột
numeric_cols = df.select_dtypes(include=['number']).columns # chọn các cột dữ liệu dạng số

for col in numeric_cols:
    if df[col].isna().all(): # cột rỗng --> skip
        continue

    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col:<25} {min_val} - {max_val}")

latitude                  10.41138 - 10.995955
longitude                 106.511271 - 107.136224
temperature (C degree)    18.5 - 38.9
humidity (%)              23 - 100
precipitation (mm)        0.0 - 24.7
windspeed (km/h)          0.0 - 37.6
winddirection (degrees)   1 - 360
cloudcover (%)            0 - 100
pressure (hPa)            999.9 - 1019.2
dewpoint (C degree)       9.4 - 28.0


--- KHOẢNG GIÁ TRỊ THAM CHIẾU ---
- latitude: -90 đến 90
- longitude: -180 đến 180
- temperature: Mức thấp kỷ lục (-89.2°C) đến Mức cao kỷ lục (56.7°C)
- humidity: 0% đến 100%
- precipitation: >= 0 mm
- windspeed: >= 0 km/h
- winddirection: 0° đến 360°
- cloudcover: 0% đến 100%
- pressure: 870 hPa đến 1085 hPa

So sánh với khoảng giá trị tham chiếu, các thuộc tính đều hợp lý, không có điểm bất thường / giá trị ngoại lai

In [66]:
# xem xét các giá trị lat/long
locations = df[['latitude', 'longitude']].drop_duplicates()

print(f"Số lượng cặp (lat, long): {len(locations)}")
print("\nChi tiết các tọa độ:")
print(locations)

Số lượng cặp (lat, long): 8

Chi tiết các tọa độ:
         latitude   longitude
0       10.775388  106.702825
16056   10.839823  106.670396
32112   10.995955  106.511271
48168   10.721314  106.546622
64224   10.475805  106.865040
80280   10.411380  107.136224
96336   10.904439  106.782443
112392  10.738544  106.729361


In [67]:
# tìm những điểm dữ liệu trùng lặp về các điều kiện thời tiết nhưng có giá trị precipitation và toạ độ khác nhau
print("\nCác điểm dữ liệu trùng lặp về các điều kiện thời tiết nhưng có giá trị precipitation và toạ độ khác nhau")
print("(latitude, longitude) : precipitation\n")

check_cols = []
for name in df.columns:
    if name not in ['latitude', 'longitude', 'precipitation (mm)']:
        check_cols.append(name)
        
check_precipitation_df = df.groupby(check_cols)
count = 0 
for name, group in check_precipitation_df:
    if group["precipitation (mm)"].nunique() > 1:
        count += 1

        long = group["longitude"].values
        lat = group["latitude"].values
        prec = group["precipitation (mm)"].values
        for i in range(len(long)):
            print(f"({lat[i]}, {long[i]}): {prec[i]}")
        print()


Các điểm dữ liệu trùng lặp về các điều kiện thời tiết nhưng có giá trị precipitation và toạ độ khác nhau
(latitude, longitude) : precipitation

(10.839823, 106.670396): 0.6
(10.995955, 106.511271): 0.9

(10.775388, 106.702825): 3.2
(10.738544, 106.729361): 3.5

(10.775388, 106.702825): 2.6
(10.738544, 106.729361): 3.6

(10.775388, 106.702825): 0.2
(10.738544, 106.729361): 0.1



Xem xét 2 cột 'latitude' và 'longtitude', nhóm có những cân nhắc sau:

**Vấn đề**

Cần xác định vai trò của latitude và longitude trong việc dự báo mưa (nên giữ hay bỏ 2 cột này?). Liệu sự khác biệt địa lý có ảnh hưởng đến mưa/lượng mưa?

**Quyết định**

Giữ lại latitude và longitude vì có sự chênh lệch lượng mưa giữa các khu vực, cần thiết để phân tích theo khu vực và để model hoạt động chính xác

## 2. Trích xuất đặc trưng

In [68]:
# tháng và giờ
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour

# mùa khô và mùa mưa
df['season'] = df['month'].apply(lambda x: 'Rainy' if 5 <= x <= 11 else 'Dry')

# label ngày/đêm
df['day_night'] = df['hour'].apply(lambda x: 'Day' if 6 <= x < 18 else 'Night')

# đổi lat-long thành id khu vực
df['location_id'] = df.groupby(['latitude', 'longitude']).ngroup() + 1
df = df.drop(columns=['latitude', 'longitude'])

df.sample(10)

,datetime,temperature (C degree),humidity (%),precipitation (mm),windspeed (km/h),winddirection (degrees),cloudcover (%),pressure (hPa),dewpoint (C degree),month,hour,season,day_night,location_id
45532,2025-07-13 04:00:00,25.0,95,0.0,5.8,214,41,1006.9,24.1,7,4,Rainy,Night,8
69028,2024-07-19 04:00:00,23.9,99,2.1,5.7,252,100,1007.8,23.7,7,4,Rainy,Night,2
79609,2025-10-03 01:00:00,25.7,93,0.0,4.9,123,92,1010.1,24.5,10,1,Rainy,Night,2
14156,2025-08-12 20:00:00,27.2,86,0.0,8.7,232,90,1008.4,24.6,8,20,Rainy,Night,5
78332,2025-08-10 20:00:00,26.3,90,0.0,13.0,248,100,1007.6,24.5,8,20,Rainy,Night,2
127938,2025-10-09 18:00:00,26.2,87,0.1,4.5,151,87,1008.6,23.8,10,18,Rainy,Night,4
65288,2024-02-14 08:00:00,27.8,68,0.0,14.2,97,5,1015.1,21.3,2,8,Dry,Day,2
109165,2025-06-18 13:00:00,32.5,66,0.0,8.9,228,100,1007.5,25.3,6,13,Rainy,Day,7
116127,2024-06-04 15:00:00,34.2,56,0.0,6.1,267,100,1006.2,24.2,6,15,Rainy,Day,4
102228,2024-09-02 12:00:00,28.0,83,0.4,6.6,257,78,1008.0,24.9,9,12,Rainy,Day,7


In [69]:
df.to_csv('data/processed/hcm_weather_processed.csv', index=False)

## 3. Xử lý dữ liệu cho mô hình học

In [70]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# encoding
le = LabelEncoder()
df['season_encoded'] = le.fit_transform(df['season']) # Dry->0, Rainy->1
df['day_night_encoded'] = le.fit_transform(df['day_night']) # Day->0, Night->1


# chọn các feature (X) và target (y)
features = ['location_id', 'month', 'hour', 'temperature (C degree)', 
            'humidity (%)', 'pressure (hPa)', 'cloudcover (%)', 
            'windspeed (km/h)', 'season_encoded', 'day_night_encoded']

X = df[features]
y = df['precipitation (mm)'] 

# tách tập dữ liệu: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Kích thước tập train: {X_train.shape}")
print(f"Kích thước tập test: {X_test.shape}")
print(X_train.head())

Kích thước tập train: (102758, 10)
Kích thước tập test: (25690, 10)
        location_id  month  hour  temperature (C degree)  humidity (%)  \
13709             5      7     5                    25.1            90   
110057            7      7    17                    30.2            72   
81613             1      2    13                    29.1            65   
77251             2      6    19                    25.8            96   
119160            4     10     0                    25.4            91   

        pressure (hPa)  cloudcover (%)  windspeed (km/h)  season_encoded  \
13709           1005.9              87               6.6               1   
110057          1004.1             100              10.6               1   
81613           1011.8              27              34.1               0   
77251           1009.0             100               3.1               1   
119160          1011.4             100               5.4               1   

        day_night_encoded  
13

In [71]:
train_df = X_train.copy()
train_df['precipitation (mm)'] = y_train

test_df = X_test.copy()
test_df['precipitation (mm)'] = y_test

# lưu file
train_df.to_csv('data/processed/train_data.csv', index=False)
test_df.to_csv('data/processed/test_data.csv', index=False)